In [2]:
import pandas as pd

# Oh Hell

In case you aren't familiar, oh hell is a trick taking game a little bit like hearts. The major difference is that at the start of each round, each participant guesses how many tricks they will take. You only get points if you get *exactly* that many tricks. Another peculiarity of the game is that the size of the hand changes each round, starting at ten and going down by one till you are dealt just a single card. 

Let's take a look at the one card hand as a nice pocket example and try to figure out the optimal play. Let's assume you've dealt (which means you're leading) You have a binary deciscion to make- you can guess one, saying you will take the trick, or 0 saying that you won't take the trick. You have three peices of information- the card in your hand, how many other players there are, and one card you know is out of the game (one card is flipped over to select the trump at the start of each round).

To beat your card, another player would have to have a card of the same suite bigger than your card or a trump or a higher trump if the card in your hand is a trump. We can boil the question down to something more tractable- what is the probability that someone else at the table has a card that will beat your card? You should bet 1 if there is a more than 50% chance you will win the hand, otherwise you should bet 0. 

Let's specify some numbers, we'll value the face cards as 11 for the jack, 12 for the queen, etc. 

Okay, so based on all the trick taking rules we know two full suites won't take your card. Spades and clubs won't take the trick, and neither will the 2 of diamonds through the 8 of diamonds. That's a total of 33 cards (26 of the other suites, 7 of the lower cards in the same suite) that won't take the trick. There are 50 cards in the deck, and we dealt 2 of them to the other players. 

Well, for the first player we know they have a 33/50 chance of not picking a card that's bigger than our card. Assuming that's happend, the second player has a 32/49 chance of not picking a card that's bigger than our card. Multiplying those probabilities we get:

In [4]:
33/50 * 32/49

0.4310204081632653

So you should guess 0 in this scenario!

Let's see if we can generalize this solution. I'd like to know what the lowest card you should bet 1 on is for each number of players. 

In [ ]:
# I'm going to work with a numbering system for cards from 2-28 so that the non-trump number cards match their value.
# Facecards count up in value from 10, so jack = 11 ... ace = 14 etc. The lowest trump will be 15 and so on. 
# All the non-trump suites will have an equal probability, so we don't need a seperate input for suite

def get_n_takes(tcard_out_val, in_hand_val):
    
    n_takes = 27 - in_hand_val

    # If the trump card out would beat our card, we subtract 1 from the possible cards that would take
    if tcard_out_val > in_hand_val:
        n_takes = n_takes - 1
    
    return n_takes

In [ ]:
def prob_win_trick(tcard_out_val, in_hand_val, num_players):
    
    num_cards_that_take = get_n_takes(tcard_out_val, in_hand_val)
    probability_of_taking = 1
    
    for n in range(num_players - 1):
        prob_no_take = ((50 - num_cards_that_take - n) / (50 - n))
        probability_of_taking = probability_of_taking * prob_no_take
        
    return probability_of_taking

In [ ]:
values = range(2, 28) 
players = range(2, 9)
trump_card_vals = range(15, 28)
results = []

for player_num in players:
    for your_card_val in values:
        for trump_card_val in trump_card_vals:
            prob_take = prob_win_trick(trump_card_val, your_card_val, player_num)
            results.append( {
                'probability_take' : prob_take,
                'card_value' : your_card_val,
                'trump_card_val' : trump_card_val,
                'players' : player_num
            })

In [ ]:
prob_win_df = pd.DataFrame(results)
prob_win_df = prob_win_df[~(prob_win_df['trump_card_val'] == prob_win_df['card_value'])]

In [ ]:
prob_win_df

Explain what we're doing below

In [ ]:
prob_win_df['should_take'] = prob_win_df['probability_take'] > .5
only_takes = prob_win_df[prob_win_df['should_take']].sort_values(['probability_take', 'card_value'])
min_prob = only_takes.loc[only_takes.groupby(['players'])['probability_take'].idxmin().dropna()][['probability_take', 'players']].rename({'probability_take' : 'min_probability_take'}, axis = 1)
only_takes = pd.merge(only_takes, min_prob) 
lowest_prob = only_takes[only_takes['probability_take'] == only_takes['min_probability_take']].drop('min_probability_take', axis = 1)

In [ ]:
lowest_card_bet_one = lowest_prob.groupby(['players', 'card_value']).min('trump_card_val')